In [1]:
import numpy as np
import pandas as pd
from keras import models, layers, Model
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(1804874, 45)
(97320, 2)


In [4]:
train_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:
train_df = train_df[['id','comment_text','target']]

In [6]:
# set index
train_df.set_index('id', inplace=True)
test_df.set_index('id', inplace=True)

In [7]:
# y_label
train_y_label = np.where(train_df['target'] >= 0.5, 1, 0)
train_df.drop(['target'], axis=1, inplace=True)

## tokenize

In [8]:
max_words = 100000
max_len = 220

In [ ]:
tokenizer = text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['comment_text'])

# texts_to_sequences
sequences_text_train = tokenizer.texts_to_sequences(train_df['comment_text'])
sequences_text_test = tokenizer.texts_to_sequences(test_df['comment_text'])

# add padding
pad_train = sequence.pad_sequences(sequences_text_train, maxlen=max_len)
pad_test = sequence.pad_sequences(sequences_text_test, maxlen=max_len)

## embedding+lstm layer

In [ ]:
# model define
model = models.Sequential()
model.add(layers.Embedding(max_words, 128, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.5, recurrent_dropout=0.3, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.5, recurrent_dropout=0.3, return_sequences=True)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# model compile
model.compile(optimizer='adam',
             loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

# keras.callbacks
callbacks_list = [ReduceLROnPlateau(
                        monitor='val_loss', patience=2, factor=0.2),	# val_loss가 patience동안 향상되지 않으면 학습률을 0.2만큼 감소 (new_lr = lr * factor)
                    ModelCheckpoint(
                        filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
history = model.fit(pad_train, train_y_label,
                     epochs=5, batch_size=1024,
                     callbacks=callbacks_list, 
                     validation_split=0.3)

## predict test set

In [13]:
test_pred = model.predict(pad_test)

In [14]:
sample_result = pd.DataFrame()
sample_result['id'] = test_df.index
sample_result['prediction'] = test_pred

In [15]:
# save sample_submission.csv
sample_result.to_csv('submission.csv', index=False)